In [1]:
!pip install snscrape

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 KB 1.2 MB/s eta 0:00:001.1 MB/s eta 0:00:01


In [141]:
import snscrape.modules.twitter as snt
import pandas as pd
import re
from nltk import word_tokenize
import nltk
import os
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/slisowski/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [142]:
query_lexicon=pd.read_csv('/path/to/query_lexicon')

In [143]:
query_lexicon=query_lexicon['terms']

In [144]:
query_lexicon

0           commit suicide
1        jump off a bridge
2       i want to overdose
3             i’m a burden
4        i’m such a burden
              ...         
59                 anxiety
60         eating disorder
61               depressed
62    don't want this life
63                     NaN
Name: terms, Length: 64, dtype: object

In [145]:
!snscrape twitter-user

usage: snscrape twitter-user [-h] [--user-id] username
snscrape twitter-user: error: the following arguments are required: username


In [146]:
labels=[
    "query",
    "tweet_id",
    "date_time",
    "lang",
    "username",
    "user_desc",
    "tweet_content",
    "location",
    
    
    
]

In [147]:
def tweet_geo(query,limit,start_date,end_date,geocode):
   """
   Return the "labels" as a dataframe while you take the inputs of time,query and the radius within london
   """
   #empty list that will store everything
   df=[]
   for i,tweet in enumerate(snt.TwitterSearchScraper(f'{query} since:{start_date} until:{end_date} geocode:"{geocode}"').get_items()):
      
      tdf=[
         query,
         tweet.id,
         tweet.date,
         tweet.lang,
         tweet.user.username,
         tweet.user.description,
         tweet.content,
         tweet.user.location
         ]
      
      df.append(tdf)
      tdf=[]
      if i==limit-1:
        break
   pdf=pd.DataFrame(df,columns=labels)
   pdf=pdf[pdf["lang"]=='en']
   pdf['is_london']=pdf['location'].apply(lambda x : re.match('london', x, re.IGNORECASE))
   pdf=pdf[~pdf['is_london'].isna()]
   pdf=pdf[["query","tweet_id","date_time","lang","username","user_desc","tweet_content","location"]]
   return pdf

In [ ]:
from tqdm import tqdm
dfs=[]
loc = '51.505234,-0.115362, 50km'
for query in tqdm(query_lexicon[:-1]):
    
    query='"{}"'.format(query)
    df=tweet_geo(query, 1000, '2015-11-11', '2022-11-11', loc)
    file=''
    query=query.replace('"','')
    token_query=word_tokenize(query)
    if len(token_query)>1:
        file='_'.join(token_query)
        file=file+'.csv'
    else:
        file=query+'.csv'
   
    file='/path/to/'+file
    df.to_csv(file)
    dfs.append(df)

 59%|█████████████████████████▎                 | 37/63 [12:47<06:34, 15.16s/it]

In [ ]:
all_data=pd.concat(dfs)

In [ ]:
all_data=all_data.drop_duplicates(subset=['tweet_id'], keep='first')

In [ ]:
all_data=all_data.reset_index(drop=True)

In [ ]:
all_data.to_csv('/path/to/file.csv', index=False)